In [5]:
# import torch
import pandas as pd
import json,os,sqlite3

`ajId`:ID of the case,   
`ajName` is the case name,   
`ajjbqk` 基本事实,  
`pjjg` 判决,    
`qw`  全部内容,   
`writId` 文档ID,   
`writName` 文档名称   

In [17]:
def insert_by_batch(con,command,array):
	cur = con.cursor()
	for batch in [array[i:i+2000] for i in range(len(array))[::2000]]:
		cur.executemany(command,batch)
	con.commit()
con = sqlite3.connect("index.db")
cur = con.cursor()
charges = json.load(open("data/documents/common_charge.json",encoding="utf-8"))
query_related = {v for key in charges for v in charges[key][:100] if v.endswith(".json")}
with open("data/query.json",encoding="utf-8") as f:
	for i in f.readlines():
		t =  json.loads(i)
		if t["path"].endswith(".json"):
			query_related.add(t["path"])
data = []
for item in query_related:
	obj = json.load(open("data/documents/documents/{}".format(item),encoding="utf-8"))
	obj["path"] = item
	data.append(obj)
going_inserted = [(index,
				  i.get("ajName",""),
				  i.get("ajjbqk",""),
				  i.get("cpfxgc",""),
				  i.get("pjjg",""),
				  i.get("writName",""),
				  i["writId"],
				  i["path"]) for index,i in enumerate(data)]
# insert_by_batch(con,"insert or ignore into cases values (?,?,?,?,?,?,?,?)",going_inserted)

In [31]:

document = data["qw"].to_list()

In [16]:
ret = dict()
class Word:
	def __init__(self,document_id,tf):
		self.document_id = document_id
		self.tf = tf
seg = [i for i in list(jieba.cut(pd.DataFrame(data).loc[6815]["qw"],cut_all=True))]
seg_count_dict = pd.Series(seg).value_counts().to_dict()
for word in seg_count_dict:
	tf = seg_count_dict[word]
	ret[word] = Word(i,tf)


# 构建倒排索引
1. 分词和去除停用词


In [1]:
import jieba
import tqdm
from collections import defaultdict
import pickle,re
import pandas as pd
import numpy as np
import sqlite3
from scipy import sparse
from scipy.sparse.linalg import norm
from typing import List,Tuple

In [2]:
conn = sqlite3.connect("index.db")
cur = conn.cursor()
matrix = sparse.load_npz("./document_matrix.npz")
matrix = matrix.T
cur.execute("""select * from words""")
words = [word[1] for word in cur.fetchall()]
cur.execute("""select * from dc""")
word_doc_count = [dc[1] for dc in cur.fetchall()]
words_idx = {word:idx for idx,word in enumerate(words)}
documents_length = cur.execute("""select count(*) from cases""").fetchone()[0]

In [3]:
def get_documents_matrix(keys:list):
	l = []
	for key in keys:
		if key in words:
			l.append(words_idx[key])
	sql = """select document from inverse_index where word_index in ({})""".format(",".join(["?"]*len(l)) )
	cur.execute(sql,l)
	return np.array(list({i[0] for i in cur.fetchall()}))
	
def query(key:str,n:int=5):
	key_with_count = pd.Series(list(jieba.cut(key))).value_counts().to_dict()
	_x = sparse.lil_matrix((len(words),1))
	# 生成查询向量的tf-idf
	for word in key_with_count:
		if word in words:
			pos = words_idx[word]
			_x[pos,0] = key_with_count[word] * np.log( (documents_length+1) / (1 + word_doc_count[pos]))
	x = _x.tocsr()
	# 分词，利用倒排索引
	docuement_to_query = get_documents_matrix(key_with_count.keys())
	m = matrix[docuement_to_query]
	# 计算查询向量与文档向量的余弦相似度
	cosine_distance = (m@x).toarray().flatten()/(norm(m,axis=1) * norm(x))

	ind = np.argpartition(cosine_distance, -n)[-n:] # ind -> cosine
	unorder_result = cosine_distance[ind]
	result_order = np.argsort(unorder_result)
	doc_ids = docuement_to_query[ind[result_order]]
	documents = get_documents(doc_ids)
	return [{"id":i,"cosine_distance":j,"title":k[0],"fact":k[1],"verdict":k[2]} for i,j,k in zip(doc_ids,unorder_result[result_order],documents)]

def get_documents(document_ids):
	command = "select ajName as case_name,ajjbqk as basic_fact,cpfxgc||pjjg as court_verdict from cases where id in ({})"\
				.format(",".join([str(i) for i in document_ids]))
	cur.execute(command)
	return cur.fetchall()

In [18]:
pd.Series(list(jieba.cut("电子商务"))).value_counts().to_dict()

{'电子商务': 1}

In [4]:
res = query("诈骗")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\QIUQIC~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.875 seconds.
Prefix dict has been built successfully.


In [5]:
res

[{'id': 6123,
  'cosine_distance': 0.2560143064969708,
  'title': '向友元、邵乐诈骗、妨害信用卡管理一案',
  'fact': ' 茂名市电白区人民检察院指控：被告人邵乐伙同被告人钟儒登、钟儒连、邵宜鹏、蔡琼昌于2016年6月份开始在广东省云浮市郁南县新城镇世纪花园8幢601房出租屋以打电话假扮熟人猜猜我是谁的方式实施电信诈骗活动。其中邵乐拨打诈骗电话6706人次、钟儒连拨打诈骗电话2139人次、邵宜鹏拨打诈骗电话2103人次、蔡琼昌拨打诈骗电话919人次、钟儒登拨打诈骗电话892人次。2016年11月15日，公安民警在该出租屋抓获被告人邵乐、钟儒连、邵宜鹏、蔡琼昌、钟儒登，缴获公民个人信息资料74张3671条、作案手机、SIM卡、充值卡等。 被告人向友元从2015年9月开始通过网上搜索公民个人信息资料，同时以每月3000元雇请被告人向微为其对公民信息进行筛选和整理。然后以每条公民信息0.2至0.4元的价格出售公民个人信息，获利100多万元。其中邵乐诈骗团伙也向向友元购买公民个人信息。2016年11月15日，公安民警在湖北省荆门市向友元家中抓获被告人向友元、向微。民警扣押其8台电脑、4个台式电脑硬盘、12张银行卡。经证据调查，硬盘储存公民个人信息4000多万条。 被告人蔡松昌于2016年8月至今，在明知被告人邵乐实施电信诈骗的情况下，在广西桂平期间，每条0.4元价格协助邵乐汇款到中国邮政储蓄银行卡号62×××55，户名吉某（向友元母亲）处，向被告人向友元处购买公民个人信息。协助汇款次数达十几次，购买公民个人信息总金额14900元。被告人蔡松昌从2016年8月开始共拨打诈骗电话2511人次。公安机关在蔡松昌处扣押手机5部、银行卡7张等作案工具。 被告人邵瑞荣于2016年7月底，以人民币1100元的价格向被告人邵乐购买2000多条公民个人信息。然后每天下午14时至17时左右，在电白区麻岗镇的一偏僻山岭上，根据邵乐提供的公民个人信息内容，以猜猜我是谁的方式拨打诈骗电话，总共拨打诈骗电话500多人次，成功诈骗两次，并且获利人民币1900元。 被告人谢景珠于2016年7月份起在茂名市电白区麻岗镇后淡鸭屋村33号其家中以打电话冒充熟人的方式拨打诈骗电话，共拨打诈骗电话1000多人次。被告人赖广强从2016年3月

In [21]:
key_with_count = pd.Series(list(jieba.cut("计算机"))).value_counts().to_dict()